# 4.4.2 As supervised problem

In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import scipy
import sklearn
import spacy
import matplotlib.pyplot as plt
import seaborn as sns
import re
from nltk.corpus import gutenberg, stopwords
from collections import Counter

Supervised NLP requires a pre-labelled dataset for training and testing, and is generally interested in categorizing text in various ways. In this case, we are going to try to predict whether a sentence comes from _Alice in Wonderland_ by Lewis Carroll or _Persuasion_ by Jane Austen. We can use any of the supervised models we've covered previously, as long as they allow categorical outcomes. In this case, we'll try Random Forests, SVM, and KNN.

Our feature-generation approach will be something called _BoW_, or _Bag of Words_. BoW is quite simple: For each sentence, we count how many times each word appears. We will then use those counts as features.  

In [2]:
# Utility function for standard text cleaning.
def text_cleaner(text):
    # Visual inspection identifies a form of punctuation spaCy does not
    # recognize: the double dash '--'.  Better get rid of it now!
    text = re.sub(r'--',' ',text)
    text = re.sub("[\[].*?[\]]", "", text)
    text = ' '.join(text.split())
    return text
    
# Load and clean the data.
persuasion = gutenberg.raw('austen-persuasion.txt')
alice = gutenberg.raw('carroll-alice.txt')

# The Chapter indicator is idiosyncratic
persuasion = re.sub(r'Chapter \d+', '', persuasion)
alice = re.sub(r'CHAPTER .*', '', alice)
    
alice = text_cleaner(alice)
persuasion = text_cleaner(persuasion)

In [3]:
# Parse the cleaned novels. This can take a bit.
nlp = spacy.load('en')
alice_doc = nlp(alice)
persuasion_doc = nlp(persuasion)

In [4]:
# Group into sentences.
alice_sents = [[sent, "Carroll"] for sent in alice_doc.sents]
persuasion_sents = [[sent, "Austen"] for sent in persuasion_doc.sents]

# Combine the sentences from the two novels into one data frame.
sentences = pd.DataFrame(alice_sents + persuasion_sents)
sentences.head()

,0,1
0,"(Alice, was, beginning, to, get, very, tired, ...",Carroll
1,"(So, she, was, considering, in, her, own, mind...",Carroll
2,"(There, was, nothing, so, VERY, remarkable, in...",Carroll
3,"(Oh, dear, !)",Carroll
4,"(I, shall, be, late, !, ')",Carroll


Time to bag some words!  Since spaCy has already tokenized and labelled our data, we can move directly to recording how often various words occur.  We will exclude stopwords and punctuation.  In addition, in an attempt to keep our feature space from exploding, we will work with lemmas (root words) rather than the raw text terms, and we'll only use the 2000 most common words for each text.

In [5]:
# Utility function to create a list of the 2000 most common words.
def bag_of_words(text):
    
    # Filter out punctuation and stop words.
    allwords = [token.lemma_
                for token in text
                if not token.is_punct
                and not token.is_stop]
    
    # Return the most common words.
    return [item[0] for item in Counter(allwords).most_common(2000)]
    

# Creates a data frame with features for each word in our common word set.
# Each value is the count of the times the word appears in each sentence.
def bow_features(sentences, common_words):
    
    # Scaffold the data frame and initialize counts to zero.
    df = pd.DataFrame(columns=common_words)
    df['text_sentence'] = sentences[0]
    df['text_source'] = sentences[1]
    df.loc[:, common_words] = 0
    
    # Process each row, counting the occurrence of words in each sentence.
    for i, sentence in enumerate(df['text_sentence']):
        
        # Convert the sentence to lemmas, then filter out punctuation,
        # stop words, and uncommon words.
        words = [token.lemma_
                 for token in sentence
                 if (
                     not token.is_punct
                     and not token.is_stop
                     and token.lemma_ in common_words
                 )]
        
        # Populate the row with word counts.
        for word in words:
            df.loc[i, word] += 1
        
        # This counter is just to make sure the kernel didn't hang.
        if i % 500 == 0:
            print("Processing row {}".format(i))
            
    return df

# Set up the bags.
alicewords = bag_of_words(alice_doc)
persuasionwords = bag_of_words(persuasion_doc)

# Combine bags to create a set of unique words.
common_words = set(alicewords + persuasionwords)

In [7]:
# Create our data frame with features. This can take a while to run.
word_counts = bow_features(sentences, common_words)
word_counts.head()

Processing row 0
Processing row 500
Processing row 1000
Processing row 1500
Processing row 2000
Processing row 2500
Processing row 3000
Processing row 3500
Processing row 4000
Processing row 4500
Processing row 5000


,mortification,violently,class,pause,apartment,domestic,legged,spread,correct,had,...,conqueror,upper,capital,roar,gather,calm,tunnel,tooth,text_sentence,text_source
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Alice, was, beginning, to, get, very, tired, ...",Carroll
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(So, she, was, considering, in, her, own, mind...",Carroll
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(There, was, nothing, so, VERY, remarkable, in...",Carroll
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Oh, dear, !)",Carroll
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(I, shall, be, late, !, ')",Carroll


In [8]:
word_counts.to_csv('bow_features_alice_persuasion', index=False)

## Trying out BoW

Now let's give the bag of words features a whirl by trying a random forest.

In [9]:
from sklearn import ensemble
from sklearn.model_selection import train_test_split

rfc = ensemble.RandomForestClassifier()
Y = word_counts['text_source']
X = np.array(word_counts.drop(['text_sentence','text_source'], 1))

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    Y,
                                                    test_size=0.4,
                                                    random_state=0)
train = rfc.fit(X_train, y_train)

print('Training set score:', rfc.score(X_train, y_train))
print('\nTest set score:', rfc.score(X_test, y_test))

Training set score: 0.9899686520376175

Test set score: 0.8980263157894737


Holy overfitting, Batman! Overfitting is a known problem when using bag of words, since it basically involves throwing a massive number of features at a model – some of those features (in this case, word frequencies) will capture noise in the training set. Since overfitting is also a known problem with Random Forests, the divergence between training score and test score is expected.


## BoW with Logistic Regression

Let's try a technique with some protection against overfitting due to extraneous features – logistic regression with ridge regularization (from ridge regression, also called L2 regularization).

In [10]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()
train = lr.fit(X_train, y_train)
print(X_train.shape, y_train.shape)
print('Training set score:', lr.score(X_train, y_train))
print('\nTest set score:', lr.score(X_test, y_test))

(3190, 3062) (3190,)
Training set score: 0.9579937304075236

Test set score: 0.9158834586466166


Logistic regression performs a bit better than the random forest.  

# BoW with Gradient Boosting

And finally, let's see what gradient boosting can do:

In [11]:
clf = ensemble.GradientBoostingClassifier()
train = clf.fit(X_train, y_train)

print('Training set score:', clf.score(X_train, y_train))
print('\nTest set score:', clf.score(X_test, y_test))

Training set score: 0.886833855799373

Test set score: 0.8735902255639098


Looks like logistic regression is the winner, but there's room for improvement.

# Same model, new inputs

What if we feed the model a different novel by Jane Austen, like _Emma_?  Will it be able to distinguish Austen from Carroll with the same level of accuracy if we insert a different sample of Austen's writing?

First, we need to process _Emma_ the same way we processed the other data, and combine it with the Alice data:

In [12]:
# Clean the Emma data.
emma = gutenberg.raw('austen-emma.txt')
emma = re.sub(r'VOLUME \w+', '', emma)
emma = re.sub(r'CHAPTER \w+', '', emma)
emma = text_cleaner(emma)
print(emma[:100])

Emma Woodhouse, handsome, clever, and rich, with a comfortable home and happy disposition, seemed to


In [13]:
# Parse our cleaned data.
emma_doc = nlp(emma)

In [14]:
# Group into sentences.
persuasion_sents = [[sent, "Austen"] for sent in persuasion_doc.sents]
emma_sents = [[sent, "Austen"] for sent in emma_doc.sents]

# Emma is quite long, let's cut it down to the same length as Alice.
emma_sents = emma_sents[0:len(alice_sents)]

In [15]:
# Build a new Bag of Words data frame for Emma word counts.
# We'll use the same common words from Alice and Persuasion.
emma_sentences = pd.DataFrame(emma_sents)
emma_bow = bow_features(emma_sentences, common_words)

print('done')

Processing row 0
Processing row 500
Processing row 1000
Processing row 1500
done


In [16]:
# Now we can model it!
# Let's use logistic regression again.

# Combine the Emma sentence data with the Alice data from the test set.
X_Emma_test = np.concatenate((
    X_train[y_train[y_train=='Carroll'].index],
    emma_bow.drop(['text_sentence','text_source'], 1)
), axis=0)
y_Emma_test = pd.concat([y_train[y_train=='Carroll'],
                         pd.Series(['Austen'] * emma_bow.shape[0])])

# Model.
print('\nTest set score:', lr.score(X_Emma_test, y_Emma_test))
lr_Emma_predicted = lr.predict(X_Emma_test)
pd.crosstab(y_Emma_test, lr_Emma_predicted)


Test set score: 0.6976137211036539


col_0,Austen,Carroll
row_0,,
Austen,1564,105
Carroll,706,307


Well look at that!  NLP approaches are generally effective on the same type of material as they were trained on. It looks like this model is actually able to differentiate multiple works by Austen from Alice in Wonderland.  Now the question is whether the model is very good at identifying Austen, or very good at identifying Alice in Wonderland, or both...

# Challenge 0:

Recall that the logistic regression model's best performance on the test set was 93%.  See what you can do to improve performance.  Suggested avenues of investigation include: Other modeling techniques (SVM?), making more features that take advantage of the spaCy information (include grammar, phrases, POS, etc), making sentence-level features (number of words, amount of punctuation), or including contextual information (length of previous and next sentences, words repeated from one sentence to the next, etc), and anything else your heart desires.  Make sure to design your models on the test set, or use cross_validation with multiple folds, and see if you can get accuracy above 90%.  

In [65]:
# Creating copy of word_counts data frame to maintain integrity of original
word_counts2 = word_counts

# Adding in column for the number of words in each sentence
word_counts2['sent_length'] = word_counts2.text_sentence.map(lambda x: len(x))

# Creating list of number of adverbs in each sentence
sentences2 = word_counts2.text_sentence
adv_count = []
for sent in sentences2:
    advs = 0
    for token in sent:
        if token.pos_ == 'ADV':
            advs +=1
    adv_count.append(advs)

# Creating column for number of adverbs per sentence
word_counts2['adv_count'] = adv_count

# Creating list of number of verbs in each sentence
verb_count = []
for sent in sentences2:
    verb = 0
    for token in sent:
        if token.pos_ == 'VERB':
            verb +=1
    verb_count.append(verb)

word_counts2['verb_count'] = verb_count

# Creating list of number of nouns in each sentence
noun_count = []
for sent in sentences2:
    noun = 0
    for token in sent:
        if token.pos_ == 'NOUN':
            noun +=1
    noun_count.append(noun)

word_counts2['noun_count'] = noun_count


# Creating list of number of punctuation marks in each sentence
punc_count = []
for sent in sentences2:
    punc = 0
    for token in sent:
        if token.pos_ == 'PUNCT':
            punc +=1
    punc_count.append(punc)

word_counts2['punc_count'] = punc_count

# Inspecting new word counts 
word_counts2.head()

,mortification,violently,class,pause,apartment,domestic,legged,spread,correct,had,...,calm,tunnel,tooth,text_sentence,text_source,sent_length,adv_count,punc_count,verb_count,noun_count
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,"(Alice, was, beginning, to, get, very, tired, ...",Carroll,67,3,10,13,12
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,"(So, she, was, considering, in, her, own, mind...",Carroll,63,7,7,11,8
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,"(There, was, nothing, so, VERY, remarkable, in...",Carroll,33,6,4,5,2
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,"(Oh, dear, !)",Carroll,3,0,1,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,"(I, shall, be, late, !, ')",Carroll,6,0,2,2,0


In [66]:
Y = word_counts2['text_source']
X = np.array(word_counts2.drop(['text_sentence','text_source'], 1))

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    Y,
                                                    test_size=0.4,
                                                    random_state=0)
lr = LogisticRegression()
train = lr.fit(X_train, y_train)
print(X_train.shape, y_train.shape)
print('Training set score:', lr.score(X_train, y_train))
print('\nTest set score:', lr.score(X_test, y_test))

(3190, 3067) (3190,)
Training set score: 0.9630094043887147

Test set score: 0.9215225563909775


The accuracy of the test set compared to the original logistic regression model increased very slightly with the latter approach.  Including the number of words, number of verbs, adverbs, nouns, and punctuation for each sentence, increases slight the capability of our model to predict which author wrote the sentence.

# Challenge 1:
Find out whether your new model is good at identifying Alice in Wonderland vs any other work, Persuasion vs any other work, or Austen vs any other work.  This will involve pulling a new book from the Project Gutenberg corpus (print(gutenberg.fileids()) for a list) and processing it.

Record your work for each challenge in a notebook and submit it below.

## New Corpus - Milton's _Paradise Lost_
We'll test both previous novels against Milton's _Paradise Lost_. 

We will now clean and process Milton's _Paradise Lost_:

In [72]:
# Clean the Emma data.
paradise = gutenberg.raw('milton-paradise.txt')
paradise = re.sub(r'VOLUME \w+', '', paradise)
paradise = re.sub(r'CHAPTER \w+', '', paradise)
paradise = re.sub(r'Book \w+', '', paradise)
paradise = text_cleaner(paradise)
print(paradise[:100])

Of Man's first disobedience, and the fruit Of that forbidden tree whose mortal taste Brought death i


In [73]:
# Parse our cleaned data.
paradise_doc = nlp(paradise)

In [75]:
# Group into sentences.
paradise_sents = [[sent, "Milton"] for sent in paradise_doc.sents]

# Paradise is a little longer than Alice, let's cut it down to the same length as Alice.
paradise_sents = paradise_sents[0:len(alice_sents)]

In [76]:
# Build a new Bag of Words data frame for Persuasian word counts.
# We'll use the same common words from Alice and Persuasion.
paradise_sentences = pd.DataFrame(paradise_sents)
paradise_bow = bow_features(paradise_sentences, common_words)

print('done')

Processing row 0
Processing row 500
Processing row 1000
Processing row 1500
done


In [98]:
# Creating function to add counts to bow features
def bow_counts(bow_df):
    counts = []
    # Counting number of each feature per sentence
    for sent in bow_df.text_sentence:
        len_sent = len(sent)
        advs = 0
        verbs = 0
        nouns = 0
        punc = 0
        for token in sent:
            if token.pos_ == 'ADV':
                advs +=1
            elif token.pos_ == 'VERB':
                verbs +=1
            elif token.pos_ == 'NOUN':
                nouns +=1
            elif token.pos_ == 'PUNCT':
                punc +=1
        # Appending counts to list
        counts.append([len_sent, advs, verbs, nouns, punc])
    # Combining original bow_features df with counts
    df = pd.concat([bow_df, pd.DataFrame(counts, 
                    columns=['sent_length', 'adv_count', 'verb_count', 'noun_count', 'punc_count'])],
                     axis=1)
    return df

In [103]:
paradise_bow.head()

,mortification,violently,class,pause,apartment,domestic,legged,spread,correct,had,...,calm,tunnel,tooth,text_sentence,text_source,sent_length,adv_count,verb_count,noun_count,punc_count
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,"(Of, Man, 's, first, disobedience, ,, and, the...",Milton,143,4,11,27,19
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,"(And, chiefly, thou, ,, O, Spirit, ,, that, do...",Milton,95,2,11,15,18
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,"(Say, first, for, Heaven, hides, nothing, from...",Milton,57,5,7,11,7
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,"(Who, first, seduced, them, to, that, foul, re...",Milton,9,1,1,2,1
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,"(Th, ', infernal, Serpent, ;, he, it, was, who...",Milton,95,1,12,17,14


We are now ready to model:
## Milton vs. Carroll
Using a logistic regression model:

In [113]:
# Identifying variables
X_paradise = paradise_bow.drop(['text_sentence','text_source'], 1)
y_paradise = paradise_bow.text_source

alice_wc = word_counts[word_counts.text_source == 'Carroll']
X_alice = alice_wc.drop(['text_sentence','text_source'], 1)
y_alice = alice_wc.text_source

# Combine the Paradise sentence data with the Alice data from the test set.
X_pa = pd.concat([X_paradise, X_alice], 0)
y_pa = pd.concat([y_paradise, y_alice], 0)

# Split into train and test sets
X_train_pa, X_test_pa, y_train_pa, y_test_pa = train_test_split(X_pa, y_pa, test_size=0.4, random_state=0)

# Model
lr2 = LogisticRegression()
train = lr2.fit(X_train_pa, y_train_pa)
print('Train set score:', lr2.score(X_train_pa, y_train_pa))
print('\nTest set score:', lr2.score(X_test_pa, y_test_pa))
lr_pa_predicted = lr2.predict(X_test_pa)
pd.crosstab(y_test_pa, lr_pa_predicted)

Train set score: 0.9765234765234765

Test set score: 0.9169161676646707


col_0,Carroll,Milton
text_source,,
Carroll,604,43
Milton,68,621


The results were really good, the model did a great job at seperating sentences by Carroll and Milton. The features are able to identify Carroll from other works.
## Milton vs. Austen
Using a logistic regression model:

In [115]:
# Identifying variables
X_paradise = paradise_bow.drop(['text_sentence','text_source'], 1)
y_paradise = paradise_bow.text_source

persuasion_wc = word_counts[word_counts.text_source == 'Austen']
X_persuasion = persuasion_wc.drop(['text_sentence','text_source'], 1)
y_persuasion = persuasion_wc.text_source

# Combine the Paradise sentence data with the Alice data from the test set.
X_pp = pd.concat([X_paradise, X_persuasion], 0)
y_pp = pd.concat([y_paradise, y_persuasion], 0)

# Split into train and test sets
X_train_pp, X_test_pp, y_train_pp, y_test_pp = train_test_split(X_pp, y_pp, test_size=0.4, random_state=0)

# Model.
lr3 = LogisticRegression()
train = lr3.fit(X_train_pp, y_train_pp)
print('Train set score:', lr3.score(X_train_pp, y_train_pp))
print('\nTest set score:', lr3.score(X_test_pp, y_test_pp))
lr_pp_predicted = lr3.predict(X_test_pp)
pd.crosstab(y_test_pp, lr_pp_predicted)

Train set score: 0.9677115987460815

Test set score: 0.930921052631579


col_0,Austen,Milton
text_source,,
Austen,1429,43
Milton,104,552


The model did a great job, using the same features to distinguish between Austen and Milton. From these few comparisons we might assume that including these features ni a model to distinguish one author to another might be suffice. However, we should try comparing with a greater number of authors and check results.